### Merge Data

#### Merge Pediatrics data with Drug and Reaction
- Pediatrics data from 3_Pediatrics_data_merge.ipynb file
- Raw Drug file from drugcharacteristics.csv.gz (Drug)
- Reaction file from reactions.csv.gz (Meddra reaction PT)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

#### Raw Data

In [2]:
adult_report_data = pd.read_csv('../../data/adult_report_resious_reporter.csv.gz',compression='gzip', index_col=0)

/tmp/ipykernel_9504/3067837061.py:1: DtypeWarning: Columns (6,18) have mixed types. Specify dtype option on import or set low_memory=False.
  adult_report_data = pd.read_csv('../../data/adult_report_resious_reporter.csv.gz',compression='gzip', index_col=0)


In [3]:
drug_characteristics_df = pd.read_csv('../../data/openFDA_drug_event/er_tables_memory_efficient/drugcharacteristics.csv.gz',compression='gzip')

/tmp/ipykernel_9504/195607322.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  drug_characteristics_df = pd.read_csv('../../data/openFDA_drug_event/er_tables_memory_efficient/drugcharacteristics.csv.gz',compression='gzip')


In [4]:
drug_characteristics_df

,drug_administration,drug_characterization,drug_indication,medicinal_product,safetyreportid
0,Intravenous (not otherwise specified),Suspect (the drug was considered by the report...,OSTEOPOROSIS,BONIVA,10003300
1,NaN,Suspect (the drug was considered by the report...,PRODUCT USED FOR UNKNOWN INDICATION,IBUPROFEN,10003301
2,NaN,Suspect (the drug was considered by the report...,NaN,LYRICA,10003302
3,NaN,Suspect (the drug was considered by the report...,NaN,FIORINAL,10003304
4,NaN,Suspect (the drug was considered by the report...,NaN,DOXYCYCLINE HYCLATE,10003304
...,...,...,...,...,...
64177915,Oral,Suspect (the drug was considered by the report...,PRODUCT USED FOR UNKNOWN INDICATION,LETAIRIS,9999996
64177916,NaN,Suspect (the drug was considered by the report...,NaN,VENTAVIS,9999997
64177917,NaN,Suspect (the drug was considered by the report...,COR PULMONALE CHRONIC,LETAIRIS,9999997
64177918,Oral,Suspect (the drug was considered by the report...,PRODUCT USED FOR UNKNOWN INDICATION,LETAIRIS,9999998


In [5]:
reaction_df = pd.read_csv("../../data/openFDA_drug_event/er_tables_memory_efficient/reactions.csv.gz", compression='gzip', low_memory=False)

In [6]:
reaction_df

,reaction_meddrapt,reaction_outcome,safetyreportid
0,Diarrhoea,NaN,10003300
1,Arthralgia,NaN,10003300
2,Headache,NaN,10003300
3,Vomiting,NaN,10003300
4,Dyspepsia,NaN,10003301
...,...,...,...
56821719,Dizziness,Unknown,9999997
56821720,Cough,Unknown,9999997
56821721,Oedema,Unknown,9999998
56821722,Chest pain,Unknown,9999999


##### Merge Data

In [7]:
adult_data = adult_report_data.copy()

In [8]:
adult_data.shape

(1519713, 19)

In [9]:
adult_data_merge = adult_report_data\
    .merge(drug_characteristics_df, on='safetyreportid', how='inner')\
    .merge(reaction_df, on='safetyreportid', how='inner')

##### Adult Data Merge

In [10]:
adult_data_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30436709 entries, 0 to 30436708
Data columns (total 25 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   patient_custom_master_age  float64
 1   patient_onsetage           float64
 2   patient_onsetageunit       object 
 3   patient_sex                object 
 4   patient_weight             float64
 5   safetyreportid             object 
 6   lastupdate_date            int64  
 7   mostrecent_receive_date    int64  
 8   receive_date               int64  
 9   congenital_anomali         float64
 10  death                      float64
 11  disabling                  float64
 12  hospitalization            float64
 13  life_threatening           float64
 14  other                      float64
 15  serious                    object 
 16  reporter_company           object 
 17  reporter_country           object 
 18  reporter_qualification     object 
 19  drug_administration        object 
 20  

##### Missing Data

In [11]:
missing_data_merge = adult_data_merge.isna().sum().sort_values(ascending=False)
missing_data_merge

congenital_anomali           30206698
disabling                    27681821
life_threatening             27492832
reaction_outcome             26950115
death                        24404682
drug_indication              19284860
drug_administration          17829724
other                        15152201
patient_weight               14243331
hospitalization              12524642
reporter_qualification        5289857
reporter_country              4452076
reporter_company              1523616
patient_sex                    199973
medicinal_product                  60
patient_custom_master_age           0
patient_onsetageunit                0
receive_date                        0
mostrecent_receive_date             0
lastupdate_date                     0
safetyreportid                      0
patient_onsetage                    0
serious                             0
drug_characterization               0
reaction_meddrapt                   0
dtype: int64

#### Age Overview

In [12]:
# check  age unit
adult_data_merge['patient_onsetageunit'].value_counts(ascending=False)

patient_onsetageunit
Year      29706381
Day         520327
Month       183684
Decade       25997
Week           320
Name: count, dtype: int64

In [13]:
#Recheck Age after filter
adult_data_merge.loc[(adult_data_merge['patient_onsetageunit'] == 'Month'),['patient_custom_master_age','patient_onsetage','patient_onsetageunit']].sort_values(by='patient_onsetage',ascending=False)

,patient_custom_master_age,patient_onsetage,patient_onsetageunit
25660020,98.166667,1178.0,Month
25660017,98.166667,1178.0,Month
25660016,98.166667,1178.0,Month
25660013,98.166667,1178.0,Month
25660015,98.166667,1178.0,Month
...,...,...,...
15527211,21.083333,253.0,Month
15527212,21.083333,253.0,Month
15527201,21.083333,253.0,Month
15527213,21.083333,253.0,Month


In [14]:
# print(type(adult_data_merge))

In [15]:
# print(adult_data_merge)

In [16]:
# Add new column with 'Year' as the value for all rows
adult_data_merge['patient_custom_master_age_unit'] = 'Year'

# Reorder columns to place the new column after 'patient_custom_master_age'
cols = list(adult_data_merge.columns)
cols.insert(cols.index('patient_custom_master_age') + 1, cols.pop(cols.index('patient_custom_master_age_unit')))
adult_data_merge = adult_data_merge[cols]

In [17]:
adult_data_merge.isna().sum().sort_values(ascending=False)

congenital_anomali                30206698
disabling                         27681821
life_threatening                  27492832
reaction_outcome                  26950115
death                             24404682
drug_indication                   19284860
drug_administration               17829724
other                             15152201
patient_weight                    14243331
hospitalization                   12524642
reporter_qualification             5289857
reporter_country                   4452076
reporter_company                   1523616
patient_sex                         199973
medicinal_product                       60
patient_onsetageunit                     0
patient_custom_master_age                0
patient_custom_master_age_unit           0
receive_date                             0
mostrecent_receive_date                  0
lastupdate_date                          0
safetyreportid                           0
patient_onsetage                         0
serious    

In [18]:
adult_data_merge.shape

(30436709, 26)

### Data Cleaning

#### Fill NA and Unknow

In [19]:
# #fill NA in columns congenital_anomali,disabling,life_threatening,death,other
# cols = ['congenital_anomali','disabling','life_threatening','death','other','hospitalization']
# for i in cols:
#     print(adult_data_merge[i].unique())

# existing_cols = [col for col in cols if col in adult_data_merge.columns]

# for col in existing_cols:
#     adult_data_merge[col] = adult_data_merge[col].fillna(0).astype(int)
    
# # After filling na
# print('')
# print('After filling NA : \n')
# for i in cols:
#     print(adult_data_merge[i].unique())

##### fill 1:1, 2:0, nan:0

In [20]:
cols = ['congenital_anomali','disabling','life_threatening','death','other','hospitalization']
for c in cols:
    adult_data_merge[c] = pd.to_numeric(adult_data_merge[c], errors='coerce')
for c in cols:
    adult_data_merge[c] = adult_data_merge[c].map({1:1}).fillna(0).astype(int)

for c in cols:
    print(c, adult_data_merge[c].unique())

congenital_anomali [0 1]
disabling [0 1]
life_threatening [0 1]
death [0 1]
other [0 1]
hospitalization [0 1]


In [21]:
# fill unknown value in columns about patient
categorical_cols_to_fill_unknown = ['patient_sex']
for col in categorical_cols_to_fill_unknown:
    adult_data_merge[col] = adult_data_merge[col].fillna('Unknown')

In [22]:
# fill unknown value in columns about reporter
categorical_cols_to_fill_unknown = ['reporter_company', 'reporter_qualification', 'reporter_country']
for col in categorical_cols_to_fill_unknown:
    adult_data_merge[col] = adult_data_merge[col].fillna('Unknown')

In [23]:
# fill unknown value in columns about reaction
categorical_cols_to_fill_unknown = ['reaction_outcome']
for col in categorical_cols_to_fill_unknown:
    adult_data_merge[col] = adult_data_merge[col].fillna('Unknown')

In [24]:
# fill unknown value in columns drug_administration
categorical_cols_to_fill_unknown = ['drug_administration']
for col in categorical_cols_to_fill_unknown:
    adult_data_merge[col] = adult_data_merge[col].fillna('Unknown')

In [25]:
adult_data_merge.isna().sum().sort_values(ascending=False)

drug_indication                   19284860
patient_weight                    14243331
medicinal_product                       60
patient_onsetage                         0
patient_custom_master_age_unit           0
patient_sex                              0
safetyreportid                           0
lastupdate_date                          0
patient_onsetageunit                     0
patient_custom_master_age                0
receive_date                             0
mostrecent_receive_date                  0
congenital_anomali                       0
death                                    0
life_threatening                         0
other                                    0
disabling                                0
hospitalization                          0
reporter_company                         0
serious                                  0
reporter_country                         0
reporter_qualification                   0
drug_characterization                    0
drug_admini

In [26]:
adult_data_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30436709 entries, 0 to 30436708
Data columns (total 26 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   patient_custom_master_age       float64
 1   patient_custom_master_age_unit  object 
 2   patient_onsetage                float64
 3   patient_onsetageunit            object 
 4   patient_sex                     object 
 5   patient_weight                  float64
 6   safetyreportid                  object 
 7   lastupdate_date                 int64  
 8   mostrecent_receive_date         int64  
 9   receive_date                    int64  
 10  congenital_anomali              int64  
 11  death                           int64  
 12  disabling                       int64  
 13  hospitalization                 int64  
 14  life_threatening                int64  
 15  other                           int64  
 16  serious                         object 
 17  reporter_company         

#### Datetime format

In [27]:
print(adult_data_merge[['lastupdate_date', 'mostrecent_receive_date', 'receive_date']].head())
date_cols_to_convert = ['lastupdate_date', 'mostrecent_receive_date', 'receive_date']

for col in date_cols_to_convert:
    adult_data_merge[col] = pd.to_datetime(adult_data_merge[col].astype(str), format='%Y%m%d', errors='coerce')

   lastupdate_date  mostrecent_receive_date  receive_date
0         20250409                 20250327      20250327
1         20250409                 20250327      20250327
2         20250408                 20250327      20250327
3         20250409                 20250327      20250327
4         20250409                 20250327      20250327


In [28]:
print(adult_data_merge[['lastupdate_date', 'mostrecent_receive_date', 'receive_date']].head())
print(adult_data_merge.info())

  lastupdate_date mostrecent_receive_date receive_date
0      2025-04-09              2025-03-27   2025-03-27
1      2025-04-09              2025-03-27   2025-03-27
2      2025-04-08              2025-03-27   2025-03-27
3      2025-04-09              2025-03-27   2025-03-27
4      2025-04-09              2025-03-27   2025-03-27
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30436709 entries, 0 to 30436708
Data columns (total 26 columns):
 #   Column                          Dtype         
---  ------                          -----         
 0   patient_custom_master_age       float64       
 1   patient_custom_master_age_unit  object        
 2   patient_onsetage                float64       
 3   patient_onsetageunit            object        
 4   patient_sex                     object        
 5   patient_weight                  float64       
 6   safetyreportid                  object        
 7   lastupdate_date                 datetime64[ns]
 8   mostrecent_receive_date       

In [29]:
adult_data_merge.shape

(30436709, 26)

#### Droup Data

In [30]:
# Droup medicinal_product because have missing 34 rows
adult_data_merge = adult_data_merge.dropna(subset=['medicinal_product'])

In [31]:
# ลบคอลัมน์ 'patient_weight'
adult_data_merge = adult_data_merge.drop(columns=['patient_weight'])

In [32]:
adult_data_merge =adult_data_merge.dropna(subset=['drug_indication'])

In [33]:
adult_data_merge =adult_data_merge.dropna(subset=['mostrecent_receive_date'])

In [34]:
adult_data_merge.isna().sum().sort_values(ascending=False)

patient_custom_master_age         0
patient_custom_master_age_unit    0
patient_onsetage                  0
patient_onsetageunit              0
patient_sex                       0
safetyreportid                    0
lastupdate_date                   0
mostrecent_receive_date           0
receive_date                      0
congenital_anomali                0
death                             0
disabling                         0
hospitalization                   0
life_threatening                  0
other                             0
serious                           0
reporter_company                  0
reporter_country                  0
reporter_qualification            0
drug_administration               0
drug_characterization             0
drug_indication                   0
medicinal_product                 0
reaction_meddrapt                 0
reaction_outcome                  0
dtype: int64

In [35]:
missing = 30436708 - adult_data_merge.shape[0]
missing

19285044

In [36]:
adult_data_merge.shape

(11151664, 25)

#### Polyphamacy

In [37]:
# unique_drugs_per_report = ped_data_merge.groupby('safetyreportid')['medicinal_product'].nunique().reset_index()
# unique_drugs_per_report.columns = ['safetyreportid', 'unique_drug_count']

#### Save Files

In [38]:
adult_data_merge.to_csv('../../data/adult_patients_report_drug_reaction.csv.gz',compression='gzip')

In [39]:
data = pd.read_csv(
    '../../data/adult_patients_report_drug_reaction.csv.gz',
    compression='gzip',
    low_memory=False,
    index_col=0
)

sampled_data = data.sample(n=1000, random_state=42)

print("ข้อมูลสุ่ม 1,000 แถว:")
print("\nจำนวนแถวและคอลัมน์:", sampled_data.shape)
print("\nคอลัมน์ทั้งหมด:", sampled_data.columns.tolist())
display(sampled_data.head(10))


sampled_data.to_csv('../../data/sampled_adult_data.csv', index=False)
sampled_data.to_excel('../../data/sampled_adult_data.xlsx', index=False)

ข้อมูลสุ่ม 1,000 แถว:

จำนวนแถวและคอลัมน์: (1000, 25)

คอลัมน์ทั้งหมด: ['patient_custom_master_age', 'patient_custom_master_age_unit', 'patient_onsetage', 'patient_onsetageunit', 'patient_sex', 'safetyreportid', 'lastupdate_date', 'mostrecent_receive_date', 'receive_date', 'congenital_anomali', 'death', 'disabling', 'hospitalization', 'life_threatening', 'other', 'serious', 'reporter_company', 'reporter_country', 'reporter_qualification', 'drug_administration', 'drug_characterization', 'drug_indication', 'medicinal_product', 'reaction_meddrapt', 'reaction_outcome']


,patient_custom_master_age,patient_custom_master_age_unit,patient_onsetage,patient_onsetageunit,patient_sex,safetyreportid,lastupdate_date,mostrecent_receive_date,receive_date,congenital_anomali,...,serious,reporter_company,reporter_country,reporter_qualification,drug_administration,drug_characterization,drug_indication,medicinal_product,reaction_meddrapt,reaction_outcome
23771064,46.0,Year,46.0,Year,Male,7653634-5,2011-12-22,2011-08-03,2011-08-03,0,...,"The adverse event resulted in death, a life th...",Unknown,UNITED STATES,Consumer or non-health professional,Unknown,Suspect (the drug was considered by the report...,BLOOD URIC ACID,RASBURICASE,DYSPNOEA,Unknown
8590345,77.0,Year,77.0,Year,Unknown,5105523-8,2006-12-08,2006-09-05,2006-09-15,0,...,"The adverse event resulted in death, a life th...",US-JNJFOC-20060902016,UNITED STATES,Physician,Unknown,Suspect (the drug was considered by the report...,SUICIDE ATTEMPT,ACETAMINOPHEN,CARDIO-RESPIRATORY ARREST,Unknown
4274231,44.0,Year,44.0,Year,Female,4698150-8,2005-10-28,2005-06-14,2005-06-23,0,...,"The adverse event resulted in death, a life th...",2005-06-0342,Unknown,Consumer or non-health professional,Unknown,Suspect (the drug was considered by the report...,DRUG USE FOR UNKNOWN INDICATION,ATIVAN,HEADACHE,Unknown
29172241,54.0,Year,54.0,Year,Female,8481615,2014-01-27,2013-01-10,2012-03-29,0,...,"The adverse event resulted in death, a life th...",PHFR2012IE001952,IE,Physician,Oral,Concomitant (the drug was reported as being ta...,AFFECTIVE DISORDER,FLUOXETIN,Constipation,Recovered/resolved
24049351,35.0,Year,35.0,Year,Female,7715798-4,2011-12-22,2011-08-29,2011-08-29,0,...,"The adverse event resulted in death, a life th...",Unknown,UNITED STATES,Consumer or non-health professional,Oral,Suspect (the drug was considered by the report...,DEPRESSION,LATUDA,NO THERAPEUTIC RESPONSE,Unknown
13051242,79.0,Year,79.0,Year,Female,5857446-4,2009-01-09,2008-04-07,2008-08-18,0,...,The adverse event did not result in any of the...,TPA2008A00457,UNITED STATES,Physician,Oral,Suspect (the drug was considered by the report...,INSOMNIA,ROZEREM,PHOTOPSIA,Unknown
22535354,47.0,Year,47.0,Year,Female,7471098-1,2011-10-10,2011-04-28,2011-05-10,0,...,"The adverse event resulted in death, a life th...",BR-BAYER-200823304LA,BRAZIL,Consumer or non-health professional,Oral,Concomitant (the drug was reported as being ta...,OSTEOARTHRITIS,METHOTREXATE,FATIGUE,Unknown
10380931,46.0,Year,46.0,Year,Female,5363643-7,2007-10-10,2007-06-18,2007-06-18,0,...,"The adverse event resulted in death, a life th...",Unknown,UNITED STATES,Other health professional,Unknown,Suspect (the drug was considered by the report...,DIABETES MELLITUS NON-INSULIN-DEPENDENT,AVANDIA,JOINT SWELLING,Unknown
12163107,48.0,Year,48.0,Year,Female,5704457-5,2008-10-10,2008-04-02,2008-04-09,0,...,"The adverse event resulted in death, a life th...",2008SP002530,FRANCE,Consumer or non-health professional,Oral,Suspect (the drug was considered by the report...,CHOLECYSTECTOMY,DESLORATADINE,GASTROENTERITIS,Unknown
29798787,63.0,Year,63.0,Year,Male,8562422-7,2012-09-28,2012-07-23,2012-08-02,0,...,The adverse event did not result in any of the...,US-BAYER-2012-074834,UNITED STATES,Consumer or non-health professional,Unknown,Suspect (the drug was considered by the report...,PAIN,ALEVE (CAPLET),NO ADVERSE EVENT,Unknown
